# Hannah Command Line Interface - Experiments

In [1]:
!tree -L 1

//: Permission denied.
.
├── augmentation
├── config.yaml
├── datasets
├── experiment
├── hannah_config.ipynb
├── README.md
├── scripts
└── trained_models

5 directories, 3 files


The main configuration ``config.yaml` composes the configurations from preset configuration files. 

In [2]:
!cat config.yaml

//: Permission denied.
defaults:
    - base_config
    - override dataset: cifar10          # Dataset configuration name
    - override features: identity        # Feature extractor configuration name (use identity for vision datasets)
    - override model: timm_resnet18      #timm_mobilenetv3_small_100      # Neural network name (for now timm_resnet50 or timm_efficientnet_lite1)
    - override optimizer: sgd          # Optimizer config name
    - override normalizer: null          # Feature normalizer (used for quantized neural networks)
    - override module: image_classifier  # Lightning module config for the training loop (image classifier for image classification tasks)
    - override augmentation: cifar_augment
    - _self_


monitor:
  metric: val_accuracy
  direction: maximize

module:
  batch_size: 2048

trainer:
  max_epochs: 50
  precision: 16

optimizer:
  lr: 0.3


Configs are composed from the config group entries in the defaults list, and show highlevel components.
The config group defaults are set from the corrsponding yaml files in the directory with the configuration tasks.  

e.g. data augmentation is from the corresponding subdirectory.

In [3]:
!tree augmentation

//: Permission denied.
augmentation
└── cifar_augment.yaml

0 directories, 1 file


The composed configuration can be shown using the command `hannah-train --cfg job`

In [ ]:
!hannah-train --cfg job

//: Permission denied.
dataset:
  data_folder: ${hydra:runtime.cwd}/datasets/
  cls: hannah.datasets.vision.Cifar10Dataset
  dataset: cifar10
  val_percent: 0.1
features:
  _target_: torch.nn.Identity
model:
  _target_: hannah.models.timm.TimmModel
  name: resnet18
scheduler:
  _target_: torch.optim.lr_scheduler.OneCycleLR
  max_lr: ${optimizer.lr}
  pct_start: 0.3
  anneal_strategy: cos
  cycle_momentum: true
  base_momentum: 0.85
  max_momentum: 0.95
  div_factor: 25.0
  final_div_factor: 10000.0
  last_epoch: -1
optimizer:
  _target_: torch.optim.sgd.SGD
  lr: 0.3
  momentum: 0.9
  dampening: 0
  weight_decay: 0.0005
  nesterov: false
module:
  _target_: hannah.modules.vision.ImageClassifierModule
  num_workers: 0
  batch_size: 2048
  shuffle_all_dataloaders: false
trainer:
  _target_: pytorch_lightning.trainer.Trainer
  accelerator: auto
  devices: 1
  limit_train_batches: 1.0
  limit_val_batches: 1.0
  limit_test_batches: 1.0
  max_epochs: 50
  default_root_dir: .
  fast_dev_run: 

## Experiments

Experiment configurations are stored in a special 

Thy can define sweeps over the parameters. 

In [2]:
!ls experiment/

//: Permission denied.
sweep_lr.yaml  sweep_models.yaml


In [6]:
!cat experiment/sweep_models.yaml

//: Permission denied.
# @package _global_
experiment_id: sweep_models
hydra:
  mode: MULTIRUN
  sweep:
    subdir: ${model.name}
  sweeper:
    params:
      model: timm_resnet18,timm_mobilenetv3_small_075,timm_mobilenetv3_small_100,kakao_resnet8 


Experiment configs can need to be used with hannah-train and the `+experiment` switch.

In [8]:
!hannah-train +experiment=sweep_models

//: Permission denied.


[2024-02-29 14:28:40,402][HYDRA] Launching 4 jobs locally
[2024-02-29 14:28:40,402][HYDRA] 	#0 : model=timm_resnet18 +experiment=sweep_models

 **                                            **
/**                                           /**
/**       ******   *******  *******   ******  /**
/******  //////** //**///**//**///** //////** /******
/**///**  *******  /**  /** /**  /**  ******* /**///**
/**  /** **////**  /**  /** /**  /** **////** /**  /**
/**  /**//******** ***  /** ***  /**//********/**  /**
//   //  //////// ///   // ///   //  //////// //   //

[2024-02-29 14:28:43,069][hannah.utils.utils][INFO] - Environment info:
[2024-02-29 14:28:43,171][hannah.utils.utils][INFO] -   Number of GPUs: 2
[2024-02-29 14:28:43,172][hannah.utils.utils][INFO] -   CUDA version: 12.1
[2024-02-29 14:28:43,174][hannah.utils.utils][INFO] -   CUDNN version: 8907
[2024-02-29 14:28:43,175][hannah.utils.utils][INFO] -   Kernel: 4.18.0-513.11.1.el8_9.x86_64
[2024-02-29 14:28:43,175][hannah.utils.util